# Task 1

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import random
from scipy import sparse

In [ ]:

def initialize_birds(L, N):
    x_positons = np.random.uniform(0, L, N)
    y_positons = np.random.uniform(0, L, N) 

    combined = np.vstack((x_positons, y_positons)).T
    return combined

# we have to make a 2d vector of velocities
def initialize_velocities(positions, v_0):

    xes = positions[0:, 0:1]
    yes = positions[0:, 1:] * 1j
    z = xes + yes

    angles = np.angle(z)
    vx = v_0 * np.cos(angles)
    vy = v_0 * np.sin(angles)    

    combined = np.vstack((vx, vy)).T
    return combined

def find_nearest_neighbors(positons):
    pass


def new_direction(direction):
    pass


def new_velocity(velocity):
    pass

# Initialization parameters

L = 2
N = 10
r = 1
r_b = 4
v_0 = 2
a = 0.15

#  Here we initialize our birds postions and velocities
positions = initialize_birds(L, N)
velocities = initialize_velocities(positions, v_0)

# Creation of birds tree 
birds_tree = cKDTree(positions, boxsize=[L,L])
dist = birds_tree.sparse_distance_matrix(birds_tree,max_distance=r,output_type='coo_matrix') # format wyniku (ptak i, ptak j, dystans)

# dist is a sparse matrix where distances beetween birds calculated, output format is (bird_i, bird_j, distance)

orient = np.random.uniform(-np.pi, np.pi, size=N)
data = np.exp(orient[dist.col]*1j)
neigh = sparse.coo_matrix(data, (dist.row(), dist.col()), shape = dist.get_shape()) # macierz do ktorej od razu wstawiamy w miejsca niezerowe macierzy exponenty
# trzeba policzyc elementy niezerowe od ptakow -> uzyskamy info ile ptakow
# w miejsca niezerowe macierzy dist trzeba zrobic exp(ithetai) gdzie theta_i to jest kąt między vektorem prędkości ptaka a poziomem

dist_no_zero = dist != 0
non_zero_index = dist.nonzero()


#dist_copy[dist_no_zero] = 1

sum_distances = np.sum(dist, axis=0)

#test = np.exp(dist)
#print(dist)
#print(sum_distances)
print(velocities)

## Test with calculating angles

In [ ]:
positions = initialize_birds(L, N)
#print(positions)
xes = positions[0:, 0:1]
yes = positions[0:, 1:] * 1j
z = xes + yes

angles = np.angle(z)
vx = v_0 * np.cos(angles)
vy = v_0 * np.sin(angles)

vel = np.vstack((vx, vy)).T
print(vel)

# Fun with sparse matrices

In [16]:
import numpy as np
from scipy.sparse import csr_matrix

# create a 2-D representation of the matrix
A = np.array([[1, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 1],\
 [0, 0, 0, 2, 0, 0]])
print("Dense matrix representation: \n", A)

# convert to sparse matrix representation 
S = csr_matrix(A)
print("Sparse matrix: \n",S)

# convert back to 2-D representation of the matrix
B = S.todense()
print("Dense matrix: \n", B)

g = B > 0
print(g)    # Here i can input numbers in nonzero indices 
B[~g] = 8
print(B)


Dense matrix representation: 
 [[1 0 0 0 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]
Sparse matrix: 
   (0, 0)	1
  (1, 2)	2
  (1, 5)	1
  (2, 3)	2
Dense matrix: 
 [[1 0 0 0 0 0]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]
[[ True False False False False False]
 [False False  True False False  True]
 [False False False  True False False]]
[[1 8 8 8 8 8]
 [8 8 2 8 8 1]
 [8 8 8 2 8 8]]
